## Procesamiento del Lenguaje Natural 1
### Carrera de Especialización en Inteligencia Artificial - FIUBA

## Desafío N° 2
### Custom embeddings con Gensim

### 2º Bimestre 2025

### Grupo

| Autores               | E-mail                    | Nº SIU  |
|---------------------- |---------------------------|---------|
| Braian Desía          | b.desia@hotmail.com       | a1804   |


### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto.

### Librerías

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Dataset

Utilizamos un dataset de texto de reviews de peliculas extraído de Kaggle. MDB es un dataset que contienee 50k reviews de peliculas para NLP con una clasificación binaria del sentimiento de la crítica.

Más información del dataset puede encontrarse en http://ai.stanford.edu/~amaas/data/sentiment/.

In [43]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\badesia\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


In [44]:
import os

# Dataset name
os.listdir(path)

['IMDB Dataset.csv']

In [45]:
# Armar el dataset. Utilizamos "," para separar el review del sentimiento.
df = pd.read_csv(os.path.join(path, 'IMDB Dataset.csv'), sep=',')
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [46]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 50000


### Pre-procesamiento

In [47]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []

for _, row in df.iterrows():
    tokens = text_to_word_sequence(row[0])
    sentence_tokens.append(tokens)

C:\Users\badesia\AppData\Local\Temp\ipykernel_19536\1072949605.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tokens = text_to_word_sequence(row[0])


In [21]:
# Demos un vistazo
sentence_tokens[:2]

[['one',
  'of',
  'the',
  'other',
  'reviewers',
  'has',
  'mentioned',
  'that',
  'after',
  'watching',
  'just',
  '1',
  'oz',
  'episode',
  "you'll",
  'be',
  'hooked',
  'they',
  'are',
  'right',
  'as',
  'this',
  'is',
  'exactly',
  'what',
  'happened',
  'with',
  'me',
  'br',
  'br',
  'the',
  'first',
  'thing',
  'that',
  'struck',
  'me',
  'about',
  'oz',
  'was',
  'its',
  'brutality',
  'and',
  'unflinching',
  'scenes',
  'of',
  'violence',
  'which',
  'set',
  'in',
  'right',
  'from',
  'the',
  'word',
  'go',
  'trust',
  'me',
  'this',
  'is',
  'not',
  'a',
  'show',
  'for',
  'the',
  'faint',
  'hearted',
  'or',
  'timid',
  'this',
  'show',
  'pulls',
  'no',
  'punches',
  'with',
  'regards',
  'to',
  'drugs',
  'sex',
  'or',
  'violence',
  'its',
  'is',
  'hardcore',
  'in',
  'the',
  'classic',
  'use',
  'of',
  'the',
  'word',
  'br',
  'br',
  'it',
  'is',
  'called',
  'oz',
  'as',
  'that',
  'is',
  'the',
  'nicknam

### Vectorización (word2vec)

In [22]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,          # cant de palabras antes y desp de la predicha
                     vector_size=300,   # dimensionalidad de los vectores 
                     negative=20,       # cantidad de negative samples... 0 es no se usa
                     workers=-1,        # si tienen más cores pueden cambiar este valor
                     sg=1)              # modelo 0:CBOW  1:skipgram

In [24]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [25]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 50000


In [26]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 42635


Vemos que la relación cantidad de words / tamaño embedding está en el orden de 140.

### Embeddings

In [27]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 0.0
Loss after epoch 1: 0.0
Loss after epoch 2: 0.0
Loss after epoch 3: 0.0
Loss after epoch 4: 0.0
Loss after epoch 5: 0.0
Loss after epoch 6: 0.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0
Loss after epoch 10: 0.0
Loss after epoch 11: 0.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0


(0, 0)

### 4 - Ensayar

In [52]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["amazing"], topn=10)

[('toasters', 0.2472677230834961),
 ('stride', 0.23483218252658844),
 ('sensual', 0.2308262139558792),
 ('cline', 0.22364233434200287),
 ('prevail', 0.2161916047334671),
 ('miraculously', 0.2158774584531784),
 ("'put", 0.21401330828666687),
 ('jingoistic', 0.21126540005207062),
 ('ornament', 0.21089081466197968),
 ('prototypical', 0.210261270403862)]

In [53]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["disappointing"], topn=10)

[('enrico', 0.21665599942207336),
 ('busters', 0.21490000188350677),
 ('infestation', 0.2143319696187973),
 ("candidate'", 0.21313492953777313),
 ('rawness', 0.20819006860256195),
 ('cinderella', 0.20753347873687744),
 ('evigan', 0.207081139087677),
 ("'big'", 0.20440497994422913),
 ('lovett', 0.20092010498046875),
 ('128', 0.20087271928787231)]

### 5 - Visualización de agrupación de vectores

In [54]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

KeyboardInterrupt: 

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)